In [76]:

#Importar librerias? Pues claro que me importan!
import numpy as np
from numpy import random
from random import choices 
import os


In [97]:
#Code meza


#Segun lo buscado, el periodo de incubacion puede ir de los 1 a los 12 dias
#con una probabilidad del 50% que termine la incubacion a los 5 dias
#def tiempo_incubacion():
#    dias = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12]
#    peso_de_decision = [0.045, 0.045, 0.045, 0.045, 0.5, 0.045, 0.045, 0.045, 0.045, .045, 0.045, 0.045]
#    n = choices(dias, peso_de_decision)
#    return n

def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.6, 0.4]
    n = choices(estado, peso_de_decision)
    return n[0]

def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [0,1]
        peso = [0.87, 0.13] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad <= 65):
        valoracion = [0,1]
        peso = [0.99, 0.01] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso)
    return n[0]
    

In [98]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugar, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugar
            self.diasestado=0
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            
            
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni) + "," +str(self.edad)            

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def esta_sano(self):
            return (self.estado == "sano") or (self.estado == "inmune")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible()):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].estado=ciudad[self.lugar][i].estadosposibles[1]
                            ciudad[self.lugar][i].diasestado=0
                            ciudad[self.lugar][i].cambioEstado=diaactual+(random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2)
                            print(ciudad[self.lugar][i].cambioEstado)
                            self.contadorInfecciones +=1
        def transicionEstados(self, dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)
                if(me_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado <= dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = dia + random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2    #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado =dia+ random.randint(int(mediaduracion-desvduracion),int(mediaduracion+desvduracion)) #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 
                                     
                

In [99]:
class Simulador:
        def __init__(self, numpersonas,personasedificio):           
            self.ciudad=crearciudadv0(numpersonas,personasedificio)
            self.dia=0
            self.hora=0
            self.numpersonas=numpersonas
            self.numedificios=len(self.ciudad)
            
           
        def __str__(self):
            #Devuelve un string con la informacion util del simulador
            return str(self.dia) + "," + str(self.hora) + "," + str(self.numpersonas) + "," +str(self.numedificios)  
        
        #Hace que el simulador avance una hora , con sus consecuencias en las personas
        def pasar_tiempo(self,mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            for j in range(len(self.ciudad)):
                for i in range(len(self.ciudad[j])):
                    self.ciudad[j][i].infectar(0.8,self.ciudad,mediaincubacion,desvincubacion,self.dia)
            if (self.hora<23):
                self.hora+=1
            else:
                self.hora=0
                self.dia+=1
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):
                        self.ciudad[j][i].transicionEstados(self.dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion)
        #Función que devuelve el total de personas sanas o que ya pasaron la enfermedad de la ciudad
        def personas_sanas(self):
            contador_sanos=0
            for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):
                        if self.ciudad[j][i].esta_sano():
                            contador_sanos+=1
            return contador_sanos
        #Función que contagia a la primera persona (la deja sintomática), se usa para crear pacientes 0
        def contagio_fijo(self):
            self.ciudad[0][0].estado=self.ciudad[0][0].estadosposibles[3]
            self.ciudad[0][0].cambioEstado=15

In [100]:
MiSimulador=Simulador(200,1000)
print(MiSimulador)
MiSimulador.contagio_fijo()
for i in range(2000):
    '''
    Datos inventados:
    MediaIncubación:7 dias
    DesvIncubación:3 dias
    MediaDuracion:14 dias
    DescDuracion:5 dias
    '''
    MiSimulador.pasar_tiempo(7,3,14,5)
    if MiSimulador.hora==0:
        print("--------")
        print(" ")
        print(" ")
        print(MiSimulador.dia)
        print(MiSimulador)
        printearEdificio(MiSimulador.ciudad ,0)
        print(MiSimulador.personas_sanas())
        print(" ")
        print(" ")
        print("--------")
        

0,0,200,1
3.0
2.5
3.0
2.5
2.5
3.5
2.5
2.0
4.0
2.0
4.0
3.5
2.5
3.5
3.0
2.5
2.5
3.5
3.5
3.5
4.5
3.0
2.5
2.0
4.5
4.0
3.5
4.5
2.5
2.5
2.0
2.0
2.5
3.0
3.5
2.0
4.0
3.5
2.0
4.5
3.0
3.0
4.0
4.0
4.0
3.5
4.5
4.5
3.5
4.5
4.5
4.5
3.0
2.0
4.0
2.5
3.0
4.5
4.0
2.0
2.0
2.5
4.5
3.0
3.5
2.0
2.5
4.0
3.5
4.0
2.0
3.5
3.0
4.5
3.5
4.5
4.5
3.5
3.0
2.5
4.0
3.0
3.0
4.0
4.0
3.0
4.5
4.0
2.5
2.5
4.5
2.5
2.5
3.0
4.5
2.5
4.0
2.5
4.0
4.0
4.0
4.5
3.5
2.5
3.0
4.0
3.5
--------
 
 
1
1,0,200,1
0,sintomatico,0,73665066E,63
1,incubando,0,5818454Y,20
2,sano,0,68671386X,38
3,incubando,0,70771236Y,74
4,incubando,0,41684656P,86
5,sano,0,56122324R,97
6,incubando,0,74317003R,109
7,sano,0,16423784J,43
8,incubando,0,70636424C,108
9,sano,0,75582432Q,20
10,sano,0,25728367S,71
11,sano,0,42577713K,82
12,incubando,0,48414607K,59
13,sano,0,27013631Q,9
14,sano,0,31311500J,15
15,incubando,0,71418134Y,97
16,incubando,0,11067811G,76
17,sano,0,42007537S,99
18,incubando,0,54088654Z,47
19,sano,0,4875757X,35
20,incubando,0,16607044D,49
21,sano,

5.5
5.5
4.0
6.0
6.5
5.0
4.0
4.5
5.5
6.5
4.5
6.5
5.0
5.5
6.0
--------
 
 
3
3,0,200,1
0,sintomatico,0,73665066E,63
1,incubandoContagioso,0,5818454Y,20
2,incubando,0,68671386X,38
3,incubando,0,70771236Y,74
4,incubandoContagioso,0,41684656P,86
5,incubandoContagioso,0,56122324R,97
6,incubandoContagioso,0,74317003R,109
7,incubando,0,16423784J,43
8,incubandoContagioso,0,70636424C,108
9,incubando,0,75582432Q,20
10,incubando,0,25728367S,71
11,incubando,0,42577713K,82
12,incubando,0,48414607K,59
13,incubando,0,27013631Q,9
14,incubando,0,31311500J,15
15,incubandoContagioso,0,71418134Y,97
16,incubando,0,11067811G,76
17,incubandoContagioso,0,42007537S,99
18,incubando,0,54088654Z,47
19,incubando,0,4875757X,35
20,incubando,0,16607044D,49
21,incubandoContagioso,0,22157665V,49
22,incubando,0,16828816S,89
23,incubando,0,6764561P,2
24,incubando,0,47086754G,54
25,incubando,0,25062032J,65
26,incubandoContagioso,0,74417077W,67
27,incubandoContagioso,0,25383243Y,99
28,incubando,0,12016470M,25
29,incubandoCo

--------
 
 
5
5,0,200,1
0,sintomatico,0,73665066E,63
1,sintomatico,0,5818454Y,20
2,incubandoContagioso,0,68671386X,38
3,incubandoContagioso,0,70771236Y,74
4,incubandoContagioso,0,41684656P,86
5,asintomatico,0,56122324R,97
6,incubandoContagioso,0,74317003R,109
7,incubando,0,16423784J,43
8,incubandoContagioso,0,70636424C,108
9,incubandoContagioso,0,75582432Q,20
10,incubando,0,25728367S,71
11,incubando,0,42577713K,82
12,incubandoContagioso,0,48414607K,59
13,incubando,0,27013631Q,9
14,incubandoContagioso,0,31311500J,15
15,incubandoContagioso,0,71418134Y,97
16,incubandoContagioso,0,11067811G,76
17,incubandoContagioso,0,42007537S,99
18,incubandoContagioso,0,54088654Z,47
19,incubandoContagioso,0,4875757X,35
20,incubandoContagioso,0,16607044D,49
21,asintomatico,0,22157665V,49
22,incubandoContagioso,0,16828816S,89
23,incubandoContagioso,0,6764561P,2
24,incubandoContagioso,0,47086754G,54
25,incubandoContagioso,0,25062032J,65
26,incubandoContagioso,0,74417077W,67
27,asintomatico,0,25383243Y,99
2

--------
 
 
7
7,0,200,1
0,sintomatico,0,73665066E,63
1,sintomatico,0,5818454Y,20
2,asintomatico,0,68671386X,38
3,incubandoContagioso,0,70771236Y,74
4,asintomatico,0,41684656P,86
5,asintomatico,0,56122324R,97
6,asintomatico,0,74317003R,109
7,incubandoContagioso,0,16423784J,43
8,sintomatico,0,70636424C,108
9,incubandoContagioso,0,75582432Q,20
10,incubandoContagioso,0,25728367S,71
11,incubandoContagioso,0,42577713K,82
12,incubandoContagioso,0,48414607K,59
13,incubandoContagioso,0,27013631Q,9
14,incubandoContagioso,0,31311500J,15
15,incubandoContagioso,0,71418134Y,97
16,sintomatico,0,11067811G,76
17,incubandoContagioso,0,42007537S,99
18,incubandoContagioso,0,54088654Z,47
19,incubandoContagioso,0,4875757X,35
20,asintomatico,0,16607044D,49
21,asintomatico,0,22157665V,49
22,sintomatico,0,16828816S,89
23,sintomatico,0,6764561P,2
24,sintomatico,0,47086754G,54
25,incubandoContagioso,0,25062032J,65
26,sintomatico,0,74417077W,67
27,asintomatico,0,25383243Y,99
28,asintomatico,0,12016470M,25
29,asi

--------
 
 
9
9,0,200,1
0,sintomatico,0,73665066E,63
1,sintomatico,0,5818454Y,20
2,asintomatico,0,68671386X,38
3,asintomatico,0,70771236Y,74
4,asintomatico,0,41684656P,86
5,asintomatico,0,56122324R,97
6,asintomatico,0,74317003R,109
7,incubandoContagioso,0,16423784J,43
8,muerto,0,70636424C,108
9,asintomatico,0,75582432Q,20
10,sintomatico,0,25728367S,71
11,sintomatico,0,42577713K,82
12,asintomatico,0,48414607K,59
13,incubandoContagioso,0,27013631Q,9
14,asintomatico,0,31311500J,15
15,asintomatico,0,71418134Y,97
16,sintomatico,0,11067811G,76
17,sintomatico,0,42007537S,99
18,asintomatico,0,54088654Z,47
19,sintomatico,0,4875757X,35
20,asintomatico,0,16607044D,49
21,asintomatico,0,22157665V,49
22,muerto,0,16828816S,89
23,sintomatico,0,6764561P,2
24,sintomatico,0,47086754G,54
25,sintomatico,0,25062032J,65
26,sintomatico,0,74417077W,67
27,asintomatico,0,25383243Y,99
28,asintomatico,0,12016470M,25
29,asintomatico,0,22016541K,70
30,sintomatico,0,12475724H,104
31,asintomatico,0,68561206T,35
32,as

...

43,inmune,0,4817572S,4
44,inmune,0,10306120G,69
45,inmune,0,36840508M,12
46,inmune,0,15130160R,57
47,inmune,0,21841625C,3
48,inmune,0,25155085P,66
49,inmune,0,18782044Z,49
50,inmune,0,46807282M,55
51,inmune,0,71723124Q,9
52,inmune,0,38650661N,30
53,inmune,0,35326462A,59
54,inmune,0,13306276V,74
55,inmune,0,82056725Q,68
56,inmune,0,77885076S,7
57,muerto,0,54277217T,100
58,inmune,0,25175642A,73
59,inmune,0,87146303Q,41
60,inmune,0,15470537R,4
61,inmune,0,87485548B,61
62,inmune,0,48721177R,1
63,inmune,0,56126614J,13
64,inmune,0,7615034X,27
65,inmune,0,18142268Y,37
66,muerto,0,14432318W,92
67,muerto,0,27830816B,69
68,inmune,0,15637508S,44
69,muerto,0,17632746A,87
70,muerto,0,85628836C,70
71,inmune,0,44522024G,20
72,muerto,0,25425470M,68
73,inmune,0,73212642P,105
74,muerto,0,88785378L,18
75,inmune,0,13125084L,46
76,muerto,0,54145065Y,88
77,inmune,0,38537677G,49
78,inmune,0,63112036J,10
79,inmune,0,12312616A,0
80,inmune,0,14123115B,25
81,inmune,0,83423618L,97
82,inmune,0,51622727D,2
83,inmun

13,inmune,0,27013631Q,9
14,inmune,0,31311500J,15
15,inmune,0,71418134Y,97
16,inmune,0,11067811G,76
17,inmune,0,42007537S,99
18,inmune,0,54088654Z,47
19,inmune,0,4875757X,35
20,inmune,0,16607044D,49
21,inmune,0,22157665V,49
22,muerto,0,16828816S,89
23,inmune,0,6764561P,2
24,inmune,0,47086754G,54
25,inmune,0,25062032J,65
26,muerto,0,74417077W,67
27,inmune,0,25383243Y,99
28,inmune,0,12016470M,25
29,inmune,0,22016541K,70
30,inmune,0,12475724H,104
31,inmune,0,68561206T,35
32,inmune,0,57001220K,67
33,inmune,0,80117350R,15
34,muerto,0,24880866H,71
35,inmune,0,61500247H,85
36,muerto,0,45733120N,92
37,inmune,0,35738770J,99
38,inmune,0,16545507C,15
39,muerto,0,34346458Y,99
40,inmune,0,85653444H,58
41,inmune,0,76467383L,107
42,inmune,0,82683141G,44
43,inmune,0,4817572S,4
44,inmune,0,10306120G,69
45,inmune,0,36840508M,12
46,inmune,0,15130160R,57
47,inmune,0,21841625C,3
48,inmune,0,25155085P,66
49,inmune,0,18782044Z,49
50,inmune,0,46807282M,55
51,inmune,0,71723124Q,9
52,inmune,0,38650661N,30
53,inm

--------
 
 
63
63,0,200,1
0,inmune,0,73665066E,63
1,inmune,0,5818454Y,20
2,inmune,0,68671386X,38
3,inmune,0,70771236Y,74
4,inmune,0,41684656P,86
5,inmune,0,56122324R,97
6,inmune,0,74317003R,109
7,inmune,0,16423784J,43
8,muerto,0,70636424C,108
9,inmune,0,75582432Q,20
10,muerto,0,25728367S,71
11,muerto,0,42577713K,82
12,inmune,0,48414607K,59
13,inmune,0,27013631Q,9
14,inmune,0,31311500J,15
15,inmune,0,71418134Y,97
16,inmune,0,11067811G,76
17,inmune,0,42007537S,99
18,inmune,0,54088654Z,47
19,inmune,0,4875757X,35
20,inmune,0,16607044D,49
21,inmune,0,22157665V,49
22,muerto,0,16828816S,89
23,inmune,0,6764561P,2
24,inmune,0,47086754G,54
25,inmune,0,25062032J,65
26,muerto,0,74417077W,67
27,inmune,0,25383243Y,99
28,inmune,0,12016470M,25
29,inmune,0,22016541K,70
30,inmune,0,12475724H,104
31,inmune,0,68561206T,35
32,inmune,0,57001220K,67
33,inmune,0,80117350R,15
34,muerto,0,24880866H,71
35,inmune,0,61500247H,85
36,muerto,0,45733120N,92
37,inmune,0,35738770J,99
38,inmune,0,16545507C,15
39,muerto,

59,inmune,0,87146303Q,41
60,inmune,0,15470537R,4
61,inmune,0,87485548B,61
62,inmune,0,48721177R,1
63,inmune,0,56126614J,13
64,inmune,0,7615034X,27
65,inmune,0,18142268Y,37
66,muerto,0,14432318W,92
67,muerto,0,27830816B,69
68,inmune,0,15637508S,44
69,muerto,0,17632746A,87
70,muerto,0,85628836C,70
71,inmune,0,44522024G,20
72,muerto,0,25425470M,68
73,inmune,0,73212642P,105
74,muerto,0,88785378L,18
75,inmune,0,13125084L,46
76,muerto,0,54145065Y,88
77,inmune,0,38537677G,49
78,inmune,0,63112036J,10
79,inmune,0,12312616A,0
80,inmune,0,14123115B,25
81,inmune,0,83423618L,97
82,inmune,0,51622727D,2
83,inmune,0,74613627V,33
84,inmune,0,65840036Y,21
85,muerto,0,75777002Y,108
86,inmune,0,8287578B,77
87,inmune,0,30387558G,39
88,muerto,0,51867266N,103
89,inmune,0,67434347A,33
90,inmune,0,73737554Z,24
91,inmune,0,26777740J,91
92,inmune,0,2037645Y,63
93,inmune,0,81410447S,16
94,inmune,0,78772633W,62
95,inmune,0,72132605M,5
96,inmune,0,41061765A,81
97,inmune,0,73374244N,79
98,inmune,0,74824460D,7
99,inm

--------
 
 
78
78,0,200,1
0,inmune,0,73665066E,63
1,inmune,0,5818454Y,20
2,inmune,0,68671386X,38
3,inmune,0,70771236Y,74
4,inmune,0,41684656P,86
5,inmune,0,56122324R,97
6,inmune,0,74317003R,109
7,inmune,0,16423784J,43
8,muerto,0,70636424C,108
9,inmune,0,75582432Q,20
10,muerto,0,25728367S,71
11,muerto,0,42577713K,82
12,inmune,0,48414607K,59
13,inmune,0,27013631Q,9
14,inmune,0,31311500J,15
15,inmune,0,71418134Y,97
16,inmune,0,11067811G,76
17,inmune,0,42007537S,99
18,inmune,0,54088654Z,47
19,inmune,0,4875757X,35
20,inmune,0,16607044D,49
21,inmune,0,22157665V,49
22,muerto,0,16828816S,89
23,inmune,0,6764561P,2
24,inmune,0,47086754G,54
25,inmune,0,25062032J,65
26,muerto,0,74417077W,67
27,inmune,0,25383243Y,99
28,inmune,0,12016470M,25
29,inmune,0,22016541K,70
30,inmune,0,12475724H,104
31,inmune,0,68561206T,35
32,inmune,0,57001220K,67
33,inmune,0,80117350R,15
34,muerto,0,24880866H,71
35,inmune,0,61500247H,85
36,muerto,0,45733120N,92
37,inmune,0,35738770J,99
38,inmune,0,16545507C,15
39,muerto,

In [8]:
#Funcion que se encarga de generar la edad de manera aleatoria
def generador_edad():
        n = random.randint(0,110) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [9]:
#Funcion que se encarga de generar numeros aleatorios 
def generador_numeros(longitud):
    numero=0
    for i in range(0,longitud): 
        n = random.randint(0,9)
        numero=10*numero+n
    return numero


In [10]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni():
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni
    input=generador_numeros(8)
    #Se consigue que toda la secuencia de numeros sea 1 solo
    
    #Se calcula la letra que ha de ser adicionada
    valor =int(input / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = input - valor;
    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia 
    DNI=str(input)+letras[valor]
    
    return DNI

In [11]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        #print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            #print(f"contador del edificio:{contadoredificio}")
            #print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            column.append(Persona(capacidadusada,generador_edad(),edificioactual, creaDni()))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
    for i in range(len(ciudad[i])):
        print(ciudad[0][i])
    

In [12]:
ciudad=crearciudadv0(200,15)

In [13]:
printearciudad(ciudad)

        

0,sano,0,87525674W,49
1,sano,0,83824107D,25
2,sano,0,27271128M,61
3,sano,0,36886460A,103
4,sano,0,3547761B,22
5,sano,0,4863717E,12
6,sano,0,24032068N,33
7,sano,0,35835825P,54
8,sano,0,25201288G,13
9,sano,0,36307017E,41
10,sano,0,3275026X,64
11,sano,0,17057407D,63
12,sano,0,2116546V,74
13,sano,0,32033302Y,54
14,sano,0,80058307E,7
15,sano,0,83217708M,24
16,sano,0,60300710T,45
17,sano,0,20552283N,60
18,sano,0,10785642E,21
19,sano,0,1768327H,25
20,sano,0,4523823E,70

21,sano,1,72621664S,46
22,sano,1,45721076C,40
23,sano,1,24767230W,49
24,sano,1,4286433S,65
25,sano,1,67315384L,48
26,sano,1,87341373T,49
27,sano,1,30880262W,38
28,sano,1,35815505C,14
29,sano,1,31080121Z,40
30,sano,1,25484220J,55
31,sano,1,51042808N,24
32,sano,1,65274044K,96
33,sano,1,88431118M,25
34,sano,1,21222784V,26
35,sano,1,87075828J,21
36,sano,1,14854208A,10
37,sano,1,66711103V,88
38,sano,1,83004780B,9
39,sano,1,73354381K,45
40,sano,1,45020035L,83
41,sano,1,22764745N,86
42,sano,1,13181027A,95
43,sano,1,26488661K,34
44,sa

In [14]:
ciudad[0][0].estado=ciudad[0][0].estadosposibles[3]
ciudad[0][0].infectar(0.8,ciudad,7.5,3.5,0)


In [49]:
printearEdificio(ciudad, 0)

0,sintomatico,0,85138111T,85
1,sano,0,53428610R,67
2,sano,0,31701017W,106
3,sano,0,50184536P,39
4,sano,0,21375251V,27
5,sano,0,75322885R,29
6,sano,0,50364662K,11
7,incubando,0,56043786P,99
8,sano,0,63444245X,106
9,sano,0,71003070T,26
10,sano,0,47112854E,6
11,sano,0,35547548J,67
12,sano,0,83618337C,43
13,sano,0,66250372E,50
14,sano,0,53422433B,94
15,sano,0,40577753A,46
16,sano,0,50334101G,78
17,sano,0,50484685F,16
18,sano,0,82252530E,14
19,sano,0,55182062W,50
20,sano,0,33411518Q,85
21,sano,0,44615778X,38
22,sano,0,16761113R,94
23,sano,0,41543538H,69
24,sano,0,17845112X,58
25,sano,0,21585284Z,38
26,sano,0,86130451M,86
27,sano,0,75763751A,89
28,sano,0,13244114R,43
29,sano,0,41813138N,94
30,sano,0,4845364T,58
31,sano,0,35352085G,82
32,sano,0,16323661D,46
33,sano,0,38528456Y,6
34,sano,0,6283447P,63
35,sano,0,68418024Q,64
